In [1]:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
  
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn import metrics
import talib as ta
from finta import TA

import warnings
warnings.filterwarnings('ignore')

In [21]:
# the pandas webreader provides remote data access to apis
import pandas_datareader as webreader

date_today = "2022-11-14"
#AKA as far back as fear greed index will go
date_start = "2018-02-01"

# set the symbol to bitcoin-usd quotes
# for more symbols check yahoo.finance.com
symbol = "BTC-USD"

In [22]:
# now we will send the request to the yahoo finance api endpoint
df = webreader.DataReader(symbol, start=date_start, end=date_today, data_source="yahoo")
df.head(5)

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2018-02-01,10288.799805,8812.280273,10237.299805,9170.540039,9959400448,9170.540039
2018-02-02,9142.280273,7796.490234,9142.280273,8830.750000,12726899712,8830.750000
2018-02-03,9430.750000,8251.629883,8852.120117,9174.910156,7263790080,9174.910156
2018-02-04,9334.870117,8031.220215,9175.700195,8277.009766,7073549824,8277.009766
2018-02-05,8364.839844,6756.680176,8270.540039,6955.270020,9285289984,6955.270020


In [23]:
df['50 SMA'] = ta.SMA(df['Close'], timeperiod=50)
df['200 SMA'] = ta.SMA(df['Close'], timeperiod=200)
df['10 EMA'] = ta.EMA(df['Close'], timeperiod=10)
df['20 EMA'] = ta.EMA(df['Close'], timeperiod=20)
df['RSI'] = ta.RSI(df['Close'], timeperiod=14)

In [24]:
df.tail()

,High,Low,Open,Close,Volume,Adj Close,50 SMA,200 SMA,10 EMA,20 EMA,RSI
Date,,,,,,,,,,,
2022-11-10,18054.312500,15834.018555,15883.158203,17586.771484,83202283721,17586.771484,19584.083730,23547.594272,19118.173558,19554.702520,38.131300
2022-11-11,17650.943359,16543.482422,17583.251953,17034.292969,55871616488,17034.292969,19536.498574,23430.474194,18739.286178,19314.663515,35.836612
2022-11-12,17066.675781,16651.775391,17036.875000,16799.185547,29717699419,16799.185547,19486.529512,23323.882817,18386.540609,19075.094185,34.874802
2022-11-13,16920.765625,16320.634766,16799.722656,16353.365234,27209183682,16353.365234,19434.856582,23209.444038,18016.872359,18815.881904,33.062715
2022-11-14,17095.193359,15876.135742,16363.816406,16363.863281,48757501952,16363.863281,19386.091895,23092.394214,17716.325254,18582.356320,33.150799


In [25]:
FeerGreed = pd.read_csv("Feer_greed.csv")  
FeerGreed




,Date,value,value_classification
0,2022-11-14,24,Extreme Fear
1,2022-11-13,22,Extreme Fear
2,2022-11-12,21,Extreme Fear
3,2022-11-11,25,Extreme Fear
4,2022-11-10,22,Extreme Fear
...,...,...,...
1740,2018-02-05,11,Extreme Fear
1741,2018-02-04,24,Extreme Fear
1742,2018-02-03,40,Fear
1743,2018-02-02,15,Extreme Fear


In [26]:

FeerGreed.set_index('Date',inplace=True)
FeerGreed

,value,value_classification
Date,,
2022-11-14,24,Extreme Fear
2022-11-13,22,Extreme Fear
2022-11-12,21,Extreme Fear
2022-11-11,25,Extreme Fear
2022-11-10,22,Extreme Fear
...,...,...
2018-02-05,11,Extreme Fear
2018-02-04,24,Extreme Fear
2018-02-03,40,Fear


In [27]:
FeerGreed.index = pd.to_datetime(FeerGreed.index)

In [28]:
dfmerged = df.merge(FeerGreed,on='Date')

dfmerged.tail(30)

,High,Low,Open,Close,Volume,Adj Close,50 SMA,200 SMA,10 EMA,20 EMA,RSI,value,value_classification
Date,,,,,,,,,,,,,
2022-10-16,19389.603516,19068.914062,19068.914062,19268.093750,17988916650,19268.093750,19700.040508,26346.549307,19289.416759,19403.553015,46.610314,24,Extreme Fear
2022-10-17,19635.802734,19173.333984,19268.562500,19550.757812,27472552998,19550.757812,19698.719375,26216.609717,19336.933314,19417.572519,50.623054,20,Extreme Fear
2022-10-18,19666.994141,19144.769531,19550.466797,19334.416016,30580012344,19334.416016,19679.447812,26081.873574,19336.475624,19409.652852,47.669923,22,Extreme Fear
2022-10-19,19348.416016,19127.687500,19335.027344,19139.535156,22425387184,19139.535156,19666.302344,25948.226504,19300.668266,19383.927357,45.116721,23,Extreme Fear
2022-10-20,19315.199219,18971.458984,19138.085938,19053.740234,24493974420,19053.740234,19646.381875,25811.227373,19255.772260,19352.480965,43.999433,23,Extreme Fear
2022-10-21,19237.384766,18770.970703,19053.203125,19172.468750,32459287866,19172.468750,19627.288437,25673.976338,19240.626168,19335.336944,45.992667,23,Extreme Fear
2022-10-22,19248.068359,19132.244141,19172.380859,19208.189453,16104440957,19208.189453,19612.056797,25542.237324,19234.728583,19323.227659,46.608395,20,Extreme Fear
2022-10-23,19646.652344,19124.197266,19207.734375,19567.007812,22128794335,19567.007812,19606.755195,25424.038672,19295.142988,19346.444817,52.470317,23,Extreme Fear
2022-10-24,19589.125000,19206.324219,19567.769531,19345.572266,30202235805,19345.572266,19593.932383,25303.247295,19304.311948,19346.361717,48.902095,22,Extreme Fear
